In [2]:
# Disease Prediction & Precaution System (English only, no external libs)
# Run: python disease_predictor.py

import tkinter as tk
from tkinter import messagebox
import re
from collections import defaultdict

# -----------------------------
# Knowledge base (expandable)
# -----------------------------
# Each disease has a canonical symptom set and an English precaution text.
DISEASE_DB = {
    "Common Cold": {
        "symptoms": {
            "cough","runny nose","sneezing","sore throat","mild fever","stuffy nose","congestion","watery eyes"
        },
        "precaution": (
            "Rest, drink warm fluids, steam inhalation, saline nasal drops, OTC lozenges. "
            "Usually resolves in 5–7 days. See a doctor if high fever >38.5°C, chest pain, "
            "or symptoms last >10 days."
        )
    },
    "Influenza (Flu)": {
        "symptoms": {
            "fever","chills","body ache","headache","dry cough","fatigue","sore throat"
        },
        "precaution": (
            "Rest, hydrate, paracetamol for fever/body ache, isolate to avoid spread. "
            "Seek medical care if breathing difficulty, persistent chest pain, confusion, "
            "or symptoms worsen after 3–4 days."
        )
    },
    "Migraine": {
        "symptoms": {
            "throbbing headache","headache","nausea","vomiting","light sensitivity","sound sensitivity","aura"
        },
        "precaution": (
            "Rest in a dark, quiet room; sleep; caffeine early may help; use prescribed pain relief. "
            "Keep a trigger diary (sleep, stress, foods). See a doctor if new/very severe headache."
        )
    },
    "Dengue": {
        "symptoms": {
            "high fever","fever","severe headache","eye pain","joint pain","muscle pain","nausea","vomiting","rash"
        },
        "precaution": (
            "Hydrate aggressively (ORS), paracetamol for fever (avoid ibuprofen/aspirin), rest. "
            "URGENT CARE if bleeding, belly pain, persistent vomiting, lethargy, or dizziness."
        )
    },
    "Typhoid": {
        "symptoms": {
            "prolonged fever","fever","abdominal pain","constipation","diarrhea","headache","loss of appetite","weakness"
        },
        "precaution": (
            "See a doctor for testing and antibiotics; drink safe water; soft diet; strict hygiene. "
            "Complete full antibiotic course if prescribed."
        )
    },
    "Malaria": {
        "symptoms": {
            "fever","chills","sweating","headache","nausea","vomiting","body ache","fatigue"
        },
        "precaution": (
            "URGENT TEST (malaria smear/rapid test) if fever with chills—especially post-mosquito exposure. "
            "Hydrate; take antimalarials only under medical supervision."
        )
    },
    "Gastroenteritis (Stomach Flu)": {
        "symptoms": {
            "loose motions","diarrhea","vomiting","nausea","abdominal cramps","stomach pain","fever","dehydration"
        },
        "precaution": (
            "ORS frequently, light diet (bananas, rice, toast), avoid dairy/spicy food, rest. "
            "See a doctor if blood in stool, severe dehydration, fever >38.5°C, or lasts >3 days."
        )
    },
    "Acid Reflux (GERD)": {
        "symptoms": {
            "heartburn","acid taste","chest burning","regurgitation","sour burps","upper abdominal discomfort","bloating"
        },
        "precaution": (
            "Small frequent meals; avoid late meals, spicy/fatty food, coffee; elevate head while sleeping; "
            "OTC antacids may help. See doctor if persistent or with weight loss/vomiting."
        )
    },
    "COVID-19": {
        "symptoms": {
            "fever","dry cough","fatigue","loss of smell","loss of taste","sore throat","headache","body ache","shortness of breath"
        },
        "precaution": (
            "Home isolate, mask, rest, hydrate, paracetamol for fever/body ache. "
            "Urgent care if oxygen saturation <94%, breathlessness, chest pain, confusion."
        )
    },
    "Sinusitis": {
        "symptoms": {
            "facial pain","facial pressure","stuffy nose","runny nose","postnasal drip","headache","reduced smell","cough"
        },
        "precaution": (
            "Steam inhalation, saline rinse, fluids, rest. See doctor if severe pain, fever, or >10 days."
        )
    },
    "Urinary Tract Infection (UTI)": {
        "symptoms": {
            "burning urination","frequent urination","urgency","lower abdominal pain","foul smelling urine","cloudy urine","fever"
        },
        "precaution": (
            "Hydrate well; avoid holding urine; see doctor for urine test and antibiotics if confirmed. "
            "Urgent care if fever with back pain (possible kidney involvement)."
        )
    },
    "Allergic Rhinitis": {
        "symptoms": {
            "sneezing","runny nose","itchy nose","watery eyes","stuffy nose","postnasal drip","cough"
        },
        "precaution": (
            "Avoid triggers (dust/pollen), rinse nose with saline, keep windows closed during high pollen, "
            "consider antihistamines after medical advice."
        )
    }
}

# Flatten a set of all known symptoms to help with parsing and hints
ALL_SYMPTOMS = sorted({s for v in DISEASE_DB.values() for s in v["symptoms"]})

# -----------------------------
# Matching logic
# -----------------------------
def normalize(text: str) -> str:
    text = text.lower().strip()
    # unify common phrasing
    replacements = {
        r'\bchest burn(ing)?\b': 'chest burning',
        r'\bstomach ache\b': 'stomach pain',
        r'\bbody aches?\b': 'body ache',
        r'\bloose motion(s)?\b': 'loose motions',
        r'\bnose block(ed)?\b': 'stuffy nose',
        r'\brunny\s*nose|nasal discharge\b': 'runny nose',
        r'\bblocked sinuses?\b': 'facial pressure',
        r'\bbreathlessness\b': 'shortness of breath',
    }
    for pat, rep in replacements.items():
        text = re.sub(pat, rep, text)
    return text

def parse_user_symptoms(raw: str):
    """
    Accepts comma/space separated text.
    Returns a set of recognized symptom tokens (best-effort matching).
    """
    raw = normalize(raw)
    # split by commas; also collect phrases that match known symptoms
    parts = [p.strip() for p in raw.split(",") if p.strip()]
    tokens = set()

    # direct exact matches first
    for p in parts:
        if p in ALL_SYMPTOMS:
            tokens.add(p)

    # fuzzy: if a known multiword symptom appears as substring
    for sym in ALL_SYMPTOMS:
        if sym in raw:
            tokens.add(sym)

    # prune to known symptoms
    tokens = {t for t in tokens if t in ALL_SYMPTOMS}
    return tokens

def score_diseases(user_syms: set):
    """
    Jaccard-based score with slight bonus for size to avoid tiny overlaps.
    Returns list of (disease, score, matched, total) sorted desc.
    """
    results = []
    for disease, info in DISEASE_DB.items():
        ds = info["symptoms"]
        inter = user_syms & ds
        if not ds:
            continue
        jaccard = len(inter) / len(ds)
        # small bonus for absolute matches count
        score = jaccard + 0.05 * len(inter)
        results.append((disease, round(score, 4), inter, len(ds)))
    results.sort(key=lambda x: x[1], reverse=True)
    return results

# -----------------------------
# GUI
# -----------------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Disease Prediction & Precaution (English)")
        self.geometry("760x560")
        self.configure(padx=14, pady=14)

        tk.Label(self, text="Enter your symptoms (comma-separated):",
                 font=("Segoe UI", 12, "bold")).pack(anchor="w")

        self.input_box = tk.Text(self, height=5, wrap="word", font=("Segoe UI", 11))
        self.input_box.pack(fill="x", pady=8)

        hint = ("Examples: fever, dry cough, fatigue  |  loose motions, vomiting  |  "
                "throbbing headache, light sensitivity  |  heartburn, acid taste")
        tk.Label(self, text=hint, fg="#555").pack(anchor="w", pady=(0, 10))

        button_frame = tk.Frame(self)
        button_frame.pack(fill="x", pady=4)

        tk.Button(button_frame, text="Predict", command=self.on_predict, width=16).pack(side="left", padx=(0,8))
        tk.Button(button_frame, text="Clear", command=self.on_clear, width=12).pack(side="left")
        tk.Button(button_frame, text="Show Known Symptoms", command=self.show_symptoms, width=22).pack(side="right")

        tk.Label(self, text="Result:", font=("Segoe UI", 12, "bold")).pack(anchor="w", pady=(14, 4))
        self.result_box = tk.Text(self, height=16, wrap="word", font=("Segoe UI", 11))
        self.result_box.pack(fill="both", expand=True)

    def on_clear(self):
        self.input_box.delete("1.0", tk.END)
        self.result_box.delete("1.0", tk.END)

    def show_symptoms(self):
        self.result_box.delete("1.0", tk.END)
        self.result_box.insert(tk.END, "Known symptoms (you can type any of these, separated by commas):\n\n")
        self.result_box.insert(tk.END, ", ".join(ALL_SYMPTOMS))

    def on_predict(self):
        raw = self.input_box.get("1.0", tk.END).strip()
        if not raw:
            messagebox.showwarning("Input needed", "Please type your symptoms, separated by commas.")
            return

        user_syms = parse_user_symptoms(raw)
        if not user_syms:
            self.result_box.delete("1.0", tk.END)
            self.result_box.insert(tk.END, "I could not recognize any symptoms from your input.\n\n"
                                           "Tip: separate symptoms with commas (e.g., fever, dry cough, fatigue).")
            return

        ranked = score_diseases(user_syms)
        top = ranked[0]
        disease, score, matched, total = top
        advice = DISEASE_DB[disease]["precaution"]

        # Compose output
        self.result_box.delete("1.0", tk.END)
        self.result_box.insert(tk.END, f"Most likely: {disease}\n")
        self.result_box.insert(tk.END, f"Matched symptoms: {', '.join(sorted(matched)) or '—'}\n")
        self.result_box.insert(tk.END, f"Confidence score: {score:.2f}\n\n")
        self.result_box.insert(tk.END, f"What to do:\n{advice}\n\n")

        # Show top 3 alternatives
        self.result_box.insert(tk.END, "Other possible conditions:\n")
        for alt in ranked[1:3]:
            d, s, m, tot = alt
            self.result_box.insert(tk.END, f"• {d}  (score {s:.2f}; matched: {', '.join(sorted(m)) or '—'})\n")

        # Safety note
        self.result_box.insert(tk.END,
            "\nNote: This tool is educational and not a medical diagnosis. "
            "If you have severe/worsening symptoms, please consult a qualified doctor."
        )

if __name__ == "__main__":
    App().mainloop()
